### Data Load

In [1]:
import pandas as pd
df = pd.read_csv("Non_liars.csv", delimiter = "|")

In [2]:
df = df.drop("Unnamed: 0", axis=1)
print(df.columns)

Index(['Unnamed: 1'], dtype='object')


In [3]:
df.shape

(96, 1)

In [4]:
a = df.values.tolist()
def concatenate_list_data(list):
    result= ''
    for element in list:
        result += str(element)
    return result

lis = []
for i in range(96):
    t = a[i]
    t = concatenate_list_data(t)
    lis.append(t)
lis

['b" Aaron Barnes Algonquin-Lake in the Hills FPD  rposes and should be used by personnel concerned with this issue, only. Because the report may contain certain developmental recommendations or special cautions concerning an , we advise that only those managers involved in the supervision of the individual have access to the report. Human Resources professionals involved in hiring decisions may also need to review the reports. Under no circumstances should the information or contents of this report be shared with the  unless under the direct supervision of a trained psychologist. We recommend that this report be considered valid for one year. While many of the characteristics of the  will not change significantly during that time, others may change enough to alter an individual\'s qualifications (e.g. , maturation, personal circumstances). Personality does change over periods of time as a result of life experiences, successes and failures over the course of life, and major life events

In [5]:
b = ' | '.join(lis)

In [6]:
import matplotlib.pyplot as plt
import gensim
import numpy as np
import spacy

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
import pyLDAvis.gensim

unable to import 'smart_open.gcs', disabling that module


In [7]:
nlp = spacy.load("en")
doc = nlp(b)

In [8]:
# for token in doc:
#     print(token.text, token.pos_, token.dep_)

### Stop words

In [9]:
my_stop_words = ['good', 'able', 'positive', 'high', 'average',
                 'major_public','paramedic','military_paramilitary',
                 'obtuse','lenient','previous','different',
                 'authoritative','accidental','annoying','violent','9ca',
                 'inoffensive','uninterested', 'personal_professional', 
                 'careless','sad','hard','deescalating','clean','nervous',
                 'impressionable','important','indifferent','northern',
                 'homesick','tall','coherent','late','illegal','reported',
                 'geographical','overwhelmed','young','primary','preliminary',
                 'new','frequent','unable','residential','relational','passive',
                 'prestigious','interfered','subordinate','married','mild',
                 'unfavorable','blunt','large','mistreated','excellent','maternal',
                 'rigorous','infrequent','needy','main','amenable','current','pertinent',
                 'inept','handwritten_complete','average_average','nuclear',
                 'standoffish','routine','divorced','inherent','polite','colored',
                 'occupied','genuine_sincere','immediate','definitive','gregarious',
                 'bright','unpleasant','slow','mock_demonstrated','unique','explanatory',
                 'bored','little','inept','lingual','excitable','prone','persistent',
                 'athletic','circular','online','isolative','perfect','frightened','lengthy',
                 'paramilitary','special_direct', 'inconsistent', 'negligent', 'possible',
                 'relative', 'authoritative', 'norther', 'strenuous', 'indifferent','impressed',
                 'hypothetical','ruptured','convolulted','rigid','inexperienced','impusive',
                 'standard','pointed','negligent','assessment','intoxicated','developmental'
                 'candid_able''required','decent','well','ideal','sizeable',
                 'sorry','recent','local','major_public','certain','complimentary','favorite',
                 'elderly','true','monotonous','hot','unscathed','worried','like','quick',
                 'extra','unsuitable','mature','bored','little','inept','lingual','excitable',
                 'prone','persistent','athletic','circular','online','isolative','perfect',
                 'frightened','lengthy','pet','america','plausible','undergraduate',
                 'boring','active','past_present','bubby','essential','reckless','expressed',
                 'unbecoming','1st','direct','public','supervisory','labor','worth','critical',
                 'volatile','domestic','surprised','rumormongering','collegiate','explicit',
                 'characteristic','alert','characteristic','sturdy','expressive','innate','teddy',
                 'mutual','insensitive','concerted','bitter','stunted','overwhelming','weekly',
                 'unscathed','ethic','tragic_traumatic','different','squalid','addictive','de',
                 'free','criminal','free','aerial','unscathed','suburban','legible','incoming',
                 'technical','exciting','underage','sick','accurate','bitter','fair','common',
                 'foremost','bi','standoffish','candid','fearful','independent','colored',
                 'outdoor','moderate','bizarre','urban','past','tentative','outstanding_standardized',
                 'coherent_proper','key','depressed','liberal','graphic','horrific','pivotal','municipal',
                 'disjointed','alcoholic','certified','incorrect','frustrating','low','current','clean',
                 'disturbing','extra','able','future','talkative','inoffensive','jumpy','attributable',
                 'sedentary','selective','stressful','shaved','bitter','ready','depressed','inflated',
                 'harsh','psychological','relaxing','verbal','irritated','bad','frank','aerobic','big',
                 'fast','amenable','tolerant','partial','preferred','second','tragic_traumatic',
                 'strong','slow','exaggerated','sizeable','dual','horrific','flippant','enlisted',
                 'crisp','prospective','ready','angry','laude','liable','productive',
                 'acceptable_favorable','adequate','complete_proper','ideal','unscathed',
                 'primary','professional','urgent','undesirable','observant','married',
                 'marked','lasting','weekly','wide','sad','industrious','inappropriate',
                 'electronic','obtuse','afraid','manual','hot','proper','southwest',
                 'heightened','unclear','incorrect','incoming','hearted','future',
                 'needy','sober','apprehensive','lucrative','complacent','cognitive',
                 'technical','sustained','hesitant','tragic','increased','medical']


### Creating bigrams and trigrams

In [10]:
for stopword in my_stop_words:
    lexeme = nlp.vocab[stopword]
    lexeme.is_stop = True

In [11]:
texts, article = [], []
for w in doc:
    # if it's not a stop word or punctuation mark, add it to our article!
    if w.text != ' | ' and not w.is_stop and not w.is_punct and not w.like_num and w.pos_ == ('ADJ' or 'NOUN'):
        # we add the lematized version of the word
        article.append(w.lemma_)
    # if it's a new line, it means we're onto our next document
    if w.text == '|':
        texts.append(article)
        article = []

In [12]:
bigram = gensim.models.Phrases(texts, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[texts], threshold=100) 

texts = [bigram[line] for line in texts]
texts = [trigram[line] for line in texts]

In [13]:
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

### Hierarchical Dirichlet Process

In [14]:
hdpmodel = HdpModel(corpus=corpus, id2word=dictionary)
hdpmodel.show_topics(num_topics=20)

/Users/akshayrajeev/opt/anaconda3/lib/python3.7/site-packages/gensim/models/hdpmodel.py:467: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  start_time = time.clock()


[(0,
  '0.009*small + 0.007*tumble + 0.007*dynamic + 0.006*suitable + 0.006*experienced + 0.005*anxious + 0.005*cognizant + 0.005*aware + 0.005*major + 0.005*involved + 0.005*approachable + 0.005*nice + 0.004*relevant + 0.004*eligible + 0.004*acceptable + 0.004*noticeable + 0.004*easygoing + 0.004*traumatic + 0.004*tempered + 0.004*noisy'),
 (1,
  '0.012*happy + 0.010*unexpected + 0.006*observable + 0.006*female + 0.006*punctual + 0.006*bright + 0.006*stocky + 0.006*distracted + 0.005*incomplete + 0.005*13th + 0.005*sloppy + 0.005*shifty + 0.004*conditional + 0.004*introductory + 0.004*apparent + 0.004*aggressive + 0.004*eager + 0.004*drunk + 0.004*orderly + 0.004*proud'),
 (2,
  '0.007*upset + 0.006*bland + 0.006*circuitous + 0.006*malicious + 0.006*regular + 0.006*financial + 0.006*kind + 0.005*implied + 0.005*intrepid + 0.005*idle + 0.005*personal + 0.005*initial + 0.005*honest + 0.005*tense + 0.005*longwinded + 0.004*stuck + 0.004*major + 0.004*correctional + 0.004*mechanical + 0.0

### Latent Dirichlet Allocation

In [15]:
ldamodel = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary)
ldamodel.show_topics()

[(0,
  '0.031*"personal" + 0.025*"acceptable" + 0.016*"major" + 0.015*"developmental" + 0.014*"traumatic" + 0.014*"fellow" + 0.014*"special" + 0.013*"present" + 0.013*"valid" + 0.013*"relevant"'),
 (1,
  '0.024*"personal" + 0.020*"acceptable" + 0.018*"physical" + 0.017*"open" + 0.015*"major" + 0.015*"developmental" + 0.014*"valid" + 0.013*"special" + 0.013*"approachable" + 0.012*"traumatic"'),
 (2,
  '0.022*"average" + 0.018*"open" + 0.016*"fellow" + 0.016*"personal" + 0.015*"developmental" + 0.015*"traumatic" + 0.013*"special" + 0.013*"capable" + 0.012*"major" + 0.012*"relevant"'),
 (3,
  '0.020*"personal" + 0.014*"developmental" + 0.012*"open" + 0.011*"fellow" + 0.011*"special" + 0.011*"traumatic" + 0.011*"complete" + 0.010*"respectful" + 0.010*"major" + 0.010*"energetic"'),
 (4,
  '0.016*"personal" + 0.014*"traumatic" + 0.012*"average" + 0.012*"developmental" + 0.011*"energetic" + 0.010*"relevant" + 0.010*"physical" + 0.010*"valid" + 0.010*"major" + 0.009*"outgoing"'),
 (5,
  '0.024

### HDP Viz

In [16]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(hdpmodel, corpus, dictionary)

/Users/akshayrajeev/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster          Freq
topic                                                   
0      0.000111  0.011757       1        1  2.428666e+01
4      0.023490  0.009147       2        1  1.508091e+01
3      0.016595  0.032776       3        1  1.404752e+01
1      0.027044 -0.010626       4        1  4.178996e+00
6      0.003889  0.036408       5        1  4.000921e+00
...         ...       ...     ...      ...           ...
145   -0.005302 -0.009207     146        1  2.638457e-20
146    0.032214 -0.010729     147        1  1.301652e-20
147   -0.003156  0.025117     148        1  6.517261e-21
148    0.014537  0.009607     149        1  3.261635e-21
149    0.000992 -0.038966     150        1  3.261635e-21

[150 rows x 5 columns], topic_info=     Category          Freq          Term      Total  loglift  logprob
771   Default  2.000000e+01       dynamic  20.000000  30.0000  30.0000
484   Default  1.900000e+01    distracted  19.000000  29.0000  29.0000
96    Default  2.000000e+01         small  20.000000  28.0000  28.0000
784   Default  1.700000e+01     cognizant  17.000000  27.0000  27.0000
405   Default  1.800000e+01        tumble  18.000000  26.0000  26.0000
..        ...           ...           ...        ...      ...      ...
412  Topic150  1.064145e-21  considerable  11.863820   1.0116  -5.4126
76   Topic150  1.057749e-21      diligent  12.200740   0.9776  -5.4186
276  Topic150  9.954618e-22        active   9.656527   1.1508  -5.4793
350  Topic150  9.788670e-22       outside  10.685641   1.0327  -5.4961
828  Topic150  9.879988e-22       married  12.058043   0.9211  -5.4868

[6365 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
557       1  0.132015      13th
557       2  0.132015      13th
557       4  0.132015      13th
557       7  0.132015      13th
127       2  0.202346      able
...     ...       ...       ...
126      11  0.069983     young
126      12  0.069983     young
258       1  0.285438  youthful
258       2  0.142719  youthful
258       3  0.142719  youthful

[3056 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 4, 2, 7, 10, 30, 11, 3, 32, 51, 8, 19, 134, 16, 42, 9, 49, 17, 6, 41, 29, 14, 13, 139, 60, 58, 44, 92, 21, 110, 38, 95, 80, 22, 23, 99, 24, 12, 91, 62, 72, 131, 125, 93, 122, 101, 129, 118, 68, 103, 15, 18, 76, 20, 25, 26, 27, 28, 31, 33, 34, 35, 36, 37, 39, 40, 43, 45, 46, 47, 48, 50, 52, 53, 54, 55, 56, 57, 59, 61, 63, 64, 65, 66, 67, 69, 70, 71, 73, 74, 75, 77, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 94, 96, 97, 98, 100, 102, 104, 105, 106, 107, 108, 109, 111, 112, 113, 114, 115, 116, 117, 119, 120, 121, 123, 124, 126, 127, 128, 130, 132, 133, 135, 136, 137, 138, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150])

### LDA Viz

In [17]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/Users/akshayrajeev/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.030797  0.036720       1        1  27.771626
1      0.038010  0.004050       2        1  22.632336
0      0.026674 -0.025371       3        1  22.042351
5      0.045680 -0.055568       4        1  12.205806
2      0.013968  0.023226       5        1   8.660708
9      0.010096  0.006040       6        1   2.749356
8     -0.048485 -0.054102       7        1   1.150232
4     -0.011788  0.032533       8        1   1.100978
3     -0.017444  0.030641       9        1   1.013138
6     -0.087508  0.001831      10        1   0.673470, topic_info=    Category        Freq           Term       Total  loglift  logprob
43   Default  189.000000       personal  189.000000  30.0000  30.0000
205  Default  110.000000        average  110.000000  29.0000  29.0000
32   Default  102.000000          major  102.000000  28.0000  28.0000
58   Default   92.000000      traumatic   92.000000  27.0000  27.0000
15   Default  110.000000  developmental  110.000000  26.0000  26.0000
..       ...         ...            ...         ...      ...      ...
58   Topic10    0.260070      traumatic   92.657913  -0.8752  -5.2440
123  Topic10    0.252867       relevant   86.840637  -0.8385  -5.2721
22   Topic10    0.246390         fellow   81.622459  -0.8025  -5.2980
150  Topic10    0.212902       outgoing   36.494446  -0.1436  -5.4441
107  Topic10    0.216712       complete   69.497864  -0.7700  -5.4264

[718 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
127       1  0.372649   able
127       2  0.124216   able
127       3  0.248433   able
127       4  0.062108   able
127       5  0.124216   able
...     ...       ...    ...
126       1  0.409341  young
126       2  0.163736  young
126       3  0.122802  young
126       4  0.245605  young
126       5  0.040934  young

[1186 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 2, 1, 6, 3, 10, 9, 5, 4, 7])

###  Finding the dominant topic in each document

In [18]:
def format_topics_sentences(ldamodel=hdpmodel, corpus=corpus, texts=lis):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=hdpmodel, corpus=corpus, texts=lis)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(15)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,2.0,0.7314,"upset, bland, circuitous, malicious, regular, ...","b"" Aaron Barnes Algonquin-Lake in the Hills FP..."
1,1,4.0,0.5501,"acceptable, personal, discernible, alert, auxi...","b"" Ryan Bachman Palos Fire Protection District..."
2,2,0.0,0.7995,"small, tumble, dynamic, suitable, experienced,...","b"" Nicholas Barnas Village of Matteson rposes..."
3,3,1.0,0.4671,"happy, unexpected, observable, female, punctua...","b"" Nicholas Bayer City of Country Club Hills ..."
4,4,4.0,0.3671,"acceptable, personal, discernible, alert, auxi...","b"" Joseph Biallas Algonquin-Lake in the Hills ..."
5,5,4.0,0.3762,"acceptable, personal, discernible, alert, auxi...","b"" Mark Bjorklund Village of Hazel Crest rpos..."
6,6,48.0,0.3953,"occupational, verbose, custodial, ample, doubt...","b"" Kiara Bogan Village of Hazel Crest rposes ..."
7,7,7.0,0.6690,"managerial, opinionated, tempered, mental, seq...","b"" Zachary Bolin Village of Schaumburg rposes..."
8,8,6.0,0.8375,"stable, problematic, collaborative, manufactur...","b"" Matthew Brannstrom Algonquin-Lake in the Hi..."
9,9,6.0,0.8504,"stable, problematic, collaborative, manufactur...","b"" Matthew Brannstrom Countryside Fire Protect..."


### Finding the most representative document for each topic

In [19]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.9880,"small, tumble, dynamic, suitable, experienced,...","b"" Gregory Butler II City of Country Club Hill..."
1,1.0,0.7063,"happy, unexpected, observable, female, punctua...","b"" Ty Green Village of Hazel Crest rposes and..."
2,2.0,0.7314,"upset, bland, circuitous, malicious, regular, ...","b"" Aaron Barnes Algonquin-Lake in the Hills FP..."
3,3.0,0.8897,"distracted, departmental, personable, confront...","b"" Chad Fedler City of Wheaton rposes and sho..."
4,4.0,0.9869,"acceptable, personal, discernible, alert, auxi...","b"" Brandon Kneller Village of Bolingbrook rpo..."


### Topic distribution across documents

In [20]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics.head(50)

,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
0.0,2.0,"upset, bland, circuitous, malicious, regular, ...",19.0,0.2000
1.0,4.0,"acceptable, personal, discernible, alert, auxi...",4.0,0.0421
2.0,0.0,"small, tumble, dynamic, suitable, experienced,...",3.0,0.0316
3.0,1.0,"happy, unexpected, observable, female, punctua...",14.0,0.1474
4.0,4.0,"acceptable, personal, discernible, alert, auxi...",19.0,0.2000
5.0,4.0,"acceptable, personal, discernible, alert, auxi...",1.0,0.0105
6.0,48.0,"occupational, verbose, custodial, ample, doubt...",6.0,0.0632
7.0,7.0,"managerial, opinionated, tempered, mental, seq...",1.0,0.0105
8.0,6.0,"stable, problematic, collaborative, manufactur...",NaN,NaN
9.0,6.0,"stable, problematic, collaborative, manufactur...",3.0,0.0316
